In [1]:
import numpy as np
import matplotlib.pyplot as plt
from importlib import reload

import Detectors.Deep_Learning.AutoEncoders.utils
reload(Detectors.Deep_Learning.AutoEncoders.utils)
from Detectors.Deep_Learning.AutoEncoders.utils import SeqDataset,train_epoch,eval_data,train_model

import flow.visualize.visualize_ring as visualize_ring
reload(visualize_ring)
from flow.visualize.visualize_ring import get_measured_leader,get_rel_dist_to_measured_leader,get_vel_of_measured_leader



import torch
print(torch.__version__)

# Anti-Flow specific functions for  detection:

from Detectors.Deep_Learning.AutoEncoders.utils import sliding_window
from Detectors.Deep_Learning.AutoEncoders.cnn_lstm_ae import CNNRecurrentAutoencoder

import utils
reload(utils)
from utils import Bando_OVM_FTL

import os

print('Libraries loaded.')


import Adversaries.controllers.car_following_adversarial
from Adversaries.controllers.car_following_adversarial import FollowerStopper_Overreact
from Adversaries.controllers.car_following_adversarial import ACC_Benign

1.7.1
Libraries loaded.


In [14]:
from flow.controllers.lane_change_controllers import AILaneChangeController


driver_controller_list_no_lc = []

sim_time = 1000
ring_length = 600


#cfm parameters:
a_mean=0.666
b_mean=21.6
s0_mean=2.21
s1_mean=2.82
Vm_mean=8.94

#lane-change parameters:

left_delta_mean = 0.5
right_delta_mean = 0.3
left_beta_mean=1.5
right_beta_mean=1.5
switching_threshold_mean = 5.0

num_human_drivers = 35

for i in range(num_human_drivers):
    a = a_mean + np.random.normal(0,0.1)
    b = b_mean + np.random.normal(0,0.5)
    s0 = s0_mean + np.random.normal(0,0.2)
    s1 = s1_mean + np.random.normal(0,0.2)
    Vm = Vm_mean + np.random.normal(0,0.5)
    
    left_delta = left_delta_mean + np.random.normal(0,0.1)
    right_delta = right_delta_mean + np.random.normal(0,0.1)
    left_beta = left_beta_mean + np.random.normal(0,0.2)
    right_beta = right_beta_mean + np.random.normal(0,0.2)
    switching_threshold = switching_threshold_mean + np.random.normal(0,0.3)

    label = 'bando_ftl_ovm_a'+str(np.round(a,2))+'_b'+str(np.round(b,2))+'_s0'+str(np.round(s0,2))+'_s1'+str(np.round(s1,2))+'_Vm'+str(np.round(Vm,2))
    cfm_controller = (Bando_OVM_FTL,{'a':a,'b':b,'s0':s0,'s1':s1,'Vm':Vm,'noise':0.1})
    
#     lc_controller = (AILaneChangeController,{'left_delta':left_delta,
#                                              'right_delta':right_delta,
#                                              left_beta:'left_beta',
#                                              right_beta:'right_beta',
#                                              switching_threshold:'switching_threshold'})
    
    driver_controller_list_no_lc.append([label,cfm_controller,1])
    
print('No lane changing.')

No lane changing.


In [19]:
from flow.controllers.lane_change_controllers import AILaneChangeController


driver_controller_list_with_lc = []

sim_time = 1000
ring_length = 600


#cfm parameters:
a_mean=0.666
b_mean=21.6
s0_mean=2.21
s1_mean=2.82
Vm_mean=8.94

#lane-change parameters:

left_delta_mean = 0.5
right_delta_mean = 0.3
left_beta_mean=1.5
right_beta_mean=1.5
switching_threshold_mean = 5.0

num_human_drivers = 35

for i in range(num_human_drivers):
    a = a_mean + np.random.normal(0,0.1)
    b = b_mean + np.random.normal(0,0.5)
    s0 = s0_mean + np.random.normal(0,0.2)
    s1 = s1_mean + np.random.normal(0,0.2)
    Vm = Vm_mean + np.random.normal(0,0.5)
    
    left_delta = left_delta_mean + np.random.normal(0,0.1)
    right_delta = right_delta_mean + np.random.normal(0,0.1)
    left_beta = left_beta_mean + np.random.normal(0,0.2)
    right_beta = right_beta_mean + np.random.normal(0,0.2)
    switching_threshold = switching_threshold_mean + np.random.normal(0,0.3)

    label = 'bando_ftl_ovm_a'+str(np.round(a,2))+'_b'+str(np.round(b,2))+'_s0'+str(np.round(s0,2))+'_s1'+str(np.round(s1,2))+'_Vm'+str(np.round(Vm,2))
    cfm_controller = (Bando_OVM_FTL,{'a':a,'b':b,'s0':s0,'s1':s1,'Vm':Vm,'noise':0.1})
    
    lc_controller = (AILaneChangeController,{'left_delta':left_delta,
                                             'right_delta':right_delta,
                                             'left_beta':left_beta,
                                             'right_beta':right_beta,
                                             'switching_threshold':switching_threshold})
    
    driver_controller_list_with_lc.append([label,cfm_controller,lc_controller,1])
    
print('With lane changing.')

With lane changing.


In [23]:
import flow
reload(flow)

#Import different needed quantities from Flow:
from flow.networks.ring import RingNetwork
from flow.core.params import VehicleParams
from flow.controllers.car_following_models import IDMController #Human driving model
from flow.controllers.routing_controllers import ContinuousRouter #Router that keeps vehicles on the ring-road

from flow.networks.ring import ADDITIONAL_NET_PARAMS
from flow.core.params import NetParams
from flow.core.params import InitialConfig
from flow.core.params import TrafficLightParams
from flow.envs.ring.accel import AccelEnv
from flow.envs.ring.accel import ADDITIONAL_ENV_PARAMS
from flow.core.params import SumoParams
from flow.core.params import EnvParams
from flow.core.params import SumoCarFollowingParams
from flow.core.experiment import Experiment

from sklearn.cluster import KMeans


# from Adversaries.controllers.car_following_adversarial import FollowerStopper_Overreact

from Adversaries.controllers.base_controller import BaseController


import os
import numpy as np

print('importated libraries for testing.')

importated libraries for testing.


In [24]:



def run_ring_sim_variable_cfm(ring_length=300,
    driver_controller_list=None,
    num_lanes=1,
    sim_time=500):

    #Simulation parameters:
    time_step = 0.1 #In seconds, how far each step of the simulation goes.
    emission_path = 'data' #Where csv is stored
    want_render = True #If we want SUMO to render the environment and display the simulation.
    sim_horizon = int(np.floor(sim_time/time_step)) #How many simulation steps will be taken -> Runs for 300 seconds

    #initialize the simulation using above parameters:
    traffic_lights = TrafficLightParams() #This is empty, so no traffic lights are used.
    initial_config = InitialConfig(shuffle=True,spacing="uniform", perturbation=1) #Vehicles start out evenly spaced.
    vehicles = VehicleParams() #The vehicles object will store different classes of drivers:
    sim_params = SumoParams(sim_step=time_step, render=want_render, emission_path=emission_path) #Sets the simulation time-step and where data will be recorded.
    env_params = EnvParams(additional_params=ADDITIONAL_ENV_PARAMS)
    net_params = NetParams(additional_params={'length':ring_length,
                                              'lanes':num_lanes,
                                              'speed_limit': 30,
                                              'resolution': 40})

    if(driver_controller_list is None):
        print('Running IDM.')
        num_human_drivers = 40
        #Default to the IDM if otherwise controllers not specified:
        vehicles.add("idm_driver",
            acceleration_controller=(IDMController, {'noise':0.1}),
            routing_controller=(ContinuousRouter, {}),
            car_following_params=SumoCarFollowingParams(speed_mode=0),
            num_vehicles=num_human_drivers)

    else:
        print('Number of classes of driver: '+str(len(driver_controller_list)))
        for driver in driver_controller_list:
            
            if(len(driver)==3):
                label = driver[0]
                cfm_controller = driver[1]
                num_vehicles = driver[2]

                vehicles.add(label,
                    acceleration_controller = cfm_controller,
                    routing_controller=(ContinuousRouter, {}),
                    car_following_params=SumoCarFollowingParams(speed_mode=0),
                    num_vehicles=num_vehicles)
                
            else:
                label = driver[0]
                cfm_controller = driver[1]
                lc_controller = driver[2]
                num_vehicles = driver[3]

                vehicles.add(label,
                    acceleration_controller = cfm_controller,
                    lane_change_controller = lc_controller,
                    routing_controller=(ContinuousRouter, {}),
                    car_following_params=SumoCarFollowingParams(speed_mode=0),
                    num_vehicles=num_vehicles)


    #initialize the simulation:
    flow_params = dict(
        exp_tag='ring_variable_cfm',
        env_name=AccelEnv,
        network=RingNetwork,
        simulator='traci',
        sim=sim_params,
        env=env_params,
        net=net_params,
        veh=vehicles,
        initial=initial_config,
        tls=traffic_lights,
    )

    flow_params['env'].horizon = sim_horizon
    exp = Experiment(flow_params)
    print('Running ring simulation, ring length: '+str(ring_length))
    
    sim_res_list = exp.run(1, convert_to_csv=True)
    
    return sim_res_list



sim_res_list_with_attack = run_ring_sim_variable_cfm(driver_controller_list = driver_controller_list_with_lc,
                                                     num_lanes=2,
                                                     sim_time=300)

Number of classes of driver: 35
Running ring simulation, ring length: 300


KeyError: 'lane_leaders'